## Work
1. 請比較使用不同層數以及不同 Dropout rate 對訓練的效果
- 5層 drop out rate 0.1: loss: 1.1513 - acc: 0.5884 - val_loss: 1.3102 - val_acc: 0.5331
- 5層 drop out rate 0.2(1e-3 -> le-2): loss: loss: 1.2990 - acc: 0.5402 - val_loss: 1.3352 - val_acc: 0.5267


2. 將 optimizer 改成使用 Adam 並加上適當的 dropout rate 檢視結果
- 5層 drop out rate 0.1: loss: 1.2519 - acc: 0.5506 - val_loss: 1.3725 - val_acc: 0.5127



In [1]:
import os
import keras
import itertools
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import Dropout

"""
建立神經網路，並加入 dropout layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128, 128, 64], drp_ratio=0.1):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = Dropout(drp_ratio)(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = Dropout(drp_ratio)(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [10]:
## 超參數設定
LEARNING_RATE = 1e-2
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95
Dropout_EXP = 0.2

In [11]:
results = {}
model = build_mlp(input_shape=x_train.shape[1:], drp_ratio=Dropout_EXP)
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
#optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
__________

50000/50000 [==============================] - 9s 186us/step - loss: 1.3286 - acc: 0.5288 - val_loss: 1.3295 - val_acc: 0.5275
Epoch 46/50
50000/50000 [==============================] - 9s 186us/step - loss: 1.3160 - acc: 0.5335 - val_loss: 1.3561 - val_acc: 0.5178
Epoch 47/50
50000/50000 [==============================] - 9s 187us/step - loss: 1.3134 - acc: 0.5350 - val_loss: 1.3583 - val_acc: 0.5136
Epoch 48/50
50000/50000 [==============================] - 9s 186us/step - loss: 1.3089 - acc: 0.5373 - val_loss: 1.3451 - val_acc: 0.5227
Epoch 49/50
50000/50000 [==============================] - 9s 187us/step - loss: 1.3005 - acc: 0.5408 - val_loss: 1.3691 - val_acc: 0.5118
Epoch 50/50
50000/50000 [==============================] - 10s 199us/step - loss: 1.2990 - acc: 0.5402 - val_loss: 1.3352 - val_acc: 0.5267


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""